In [2]:
import pickle
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *

In [12]:
with open("keywords.pkl", "rb") as f:
    keywords = pickle.load(f)        

In [13]:
labels_list = ['Physical and meteorological environment',
                'Soil and soil productivity',
                'Vegetation',
                'Water quality and quantity',
                'Fish and fish habitat',
                'Wetlands',
                'Wildlife and wildlife habitat',
                'Species at Risk or Species of Special Status and related habitat',
                'Greenhouse gas (GHG) emissions and climate change',
                'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
                'Air emissions',
                'Acoustic environment',
                'Electromagnetism and Corona Discharge',
                'Human occupancy and resource use',
                'Heritage resources',
                'Navigation and navigation safety',
                'Traditional land and resource use',
                'Social and cultural well-being',
                'Human health and aesthetics',
                'Infrastructure and services',
                'Employment and economy',
                'Environmental Obligations',
                'Rights of Indigenous Peoples']

In [14]:
def keywords_to_label(i, table_text, keywords_for_label, label):
    if any(word in table_text for word in keywords_for_label):
        df[f'{label}'][i] = 1
    else:
        df[f'{label}'][i] = 0

In [43]:
# df = pd.read_csv('./data/ESA_website_ENG_with_VCs_onehot.csv')
df = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv')

In [44]:
table_texts = df['text'].tolist()

tokenized_table_texts = []
stemmer = PorterStemmer()

for i, table_text in enumerate(table_texts):
    processed_text = word_tokenize(table_text)
    processed_text = [w for w in processed_text if w not in stopwords.words("english")]
    processed_text = [stemmer.stem(w) for w in processed_text]
    tokenized_table_texts.append(processed_text)
    
print(tokenized_table_texts[0])

['tabl', '3', 'summari', 'OF', 'aquat', 'field', 'work', 'and', 'aborigin', 'field', 'studi', 'particip', 'for', 'the', 'project', 'survey', 'date', 'aborigin', 'commun', 'detail', 'juli', '9', '12', '2011', 'blueberri', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'summer', '2011', 'open', 'water', 'aquat', 'assess', 'septemb', '18', 'octob', '10', '2011', 'blueberri', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'fall', '2011', 'open', 'water', 'aquat', 'assess', 'februari', '26', 'march', '12', '2012', 'blueberri', 'river', 'first', 'nation', 'doig', 'river', 'first', 'nation', 'halfway', 'river', 'first', 'nation', 'kelli', 'lake', 'cree', 'first', 'nation', 'mcleod', 'lake', 'indian', 'band', 'north', 'east', 'mti', 'associ', 'winter', '2012', 'aquat', 'habitat', 'investig']


In [46]:
with open('tokenized_table_texts.pkl', 'wb') as f:
    pickle.dump(tokenized_table_texts, f)

In [47]:
with open('tokenized_table_texts.pkl', 'rb') as f:
    tokenized_table_text = pickle.load(f)

In [48]:
for label in labels_list:
    df[f'{label}'] = 0

df.head(2)

,Unnamed: 0,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,...,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,0,0,0,0,0,0,0,0,0,0
1,1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,0,0,0,0,0,0,0,0,0,0


In [49]:
for i, table_text in enumerate(tokenized_table_texts):
    for keywords_for_label, label in zip(keywords, labels_list):
        keywords_to_label(i, table_text, keywords_for_label, label)
        
with open("esa_index_ENG_labeled.pkl", "wb") as f:
    pickle.dump(df, f)

df.head()

<ipython-input-14-e88376b0869c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{label}'][i] = 0
<ipython-input-14-e88376b0869c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{label}'][i] = 1


,Unnamed: 0,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,...,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,0,0,1,1,1,0,0,0,0,1
1,1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,0,0,1,1,1,0,1,1,1,1
2,2,9136,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,0,0,1,1,0,0,1,1,1,1
3,3,9137,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,0,0,1,1,0,0,1,1,1,1
4,4,9138,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,...,0,0,1,1,0,0,0,1,1,1


In [3]:
with open("esa_index_ENG_labeled.pkl", "rb") as f:
    df = pickle.load(f)

In [28]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 500)

In [30]:
df.iloc[:, 34:59][0:50]

,text,label,Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples
0,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT Survey Date Aboriginal Communities Detail July 9 to 12 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Summer 2011 open water aquatic assessments September 18 to October 10 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mtis Association Fall 2011 open water aquatic assessments February...,-1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1
1,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION Field Site No Name Legal Location UTM Coordinates NAD83 Zone 10 LatitudeLongitude DDMMSS Elevation m asl Fish Stream Classification 17 Fish and Wildlife Timing Window Low Risk 27 Riparian Management Area m 127 Open Water Mean Channel Morphology m 3 Fish Species Captured or Observed During Open Water Aquatic Assessment Previously Documented 4 Location of Nearest Known Fish Habitat 56 Beaver Activity...,-1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,1,1,1,1
2,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION Field Site No Name Legal Location UTM Coordinates NAD83 Zone 10 LatitudeLongitude DDMMSS Elevation m asl Fish Stream Classification 17 Fish and Wildlife Timing Window Low Risk 27 Riparian Management Area m 127 Open Water Mean Channel Morphology m 3 Fish Species Captured or Observed During Open Water Aquatic Assessment Previously Documented 4 Location of Nearest Known Fish Habitat 56 Beaver Activity...,-1,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1,1
3,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION Field Site No Name Legal Location UTM Coordinates NAD83 Zone 10 LatitudeLongitude DDMMSS Elevation m asl Fish Stream Classification 17 Fish and Wildlife Timing Window Low Risk 27 Riparian Management Area m 127 Open Water Mean Channel Morphology m 3 Fish Species Captured or Observed During Open Water Aquatic Assessment Previously Documented 4 Location of Nearest Known Fish Habitat 56 Beaver Activity...,-1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1,1
4,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION Field Site No Name Legal Location UTM Coordinates NAD83 Zone 10 LatitudeLongitude DDMMSS Elevation m asl Fish Stream Classification 17 Fish and Wildlife Timing Window Low Risk 27 Riparian Management Area m 127 Open Water Mean Channel Morphology m 3 Fish Species Captured or Observed During Open Water Aquatic Assessment Previously Documented 4 Location of Nearest Known Fish Habitat 56 Beaver Activity...,-1,1,1,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1,1
5,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG THE NORTH MONTNEY MAINLINE AITKEN CREEK SECTION Field Site No Name Legal Location UTM Coordinates NAD83 Zone 10 LatitudeLongitude DDMMSS Elevation m asl Fish Stream Classification 17 Fish and Wildlife Timing Window Low Risk 27 Riparian Management Area m 127 Open Water Mean Channel Morphology m 3 Fish Species Captured or Observed During Open Water Aquatic Assessment Previously Documented 4 Location of Nearest Known Fish Habitat 56 Beaver Activity...,-1,1,1,0,1,1,0,1,0,0,